In [1]:
!pip install transformers

In [8]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.7 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [2]:
import pandas as pd
import numpy as np
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, AutoConfig

In [3]:
model_path = "/content/drive/MyDrive/t5results/checkpoint-7000"

In [4]:
config = AutoConfig.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path, config=config)
tokenizer = AutoTokenizer.from_pretrained("t5-small")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [5]:
from datasets import load_dataset

In [6]:
dataset = load_dataset("json", data_files="/content/drive/MyDrive/qa_pairs_filtered.json")

Generating train split: 0 examples [00:00, ? examples/s]

In [10]:
train_dataset = dataset['train']
sampled_data = train_dataset.shuffle(seed=42).select(range(500))
sample_list = [{'input': row['input'], 'output': row['output']} for row in sampled_data]

In [11]:
input_list = [row['input'] for row in sampled_data]
output_list = [row['output'] for row in sampled_data]

In [12]:
print(input_list[0:5])
print(output_list[0:5])

['What was the status of the request from IP address 89.196.153.43 to /64746/productModel/200x200 on 2019-01-23 21:50:36?', 'When did IP address 81.163.1.201 access /126/brand?', 'When did IP address 77.104.78.66 access /56198/productModel/100x100?', 'Which IP addresses experienced a success when accessing /13903/productModel/200x200?', 'How many times was /23951/productModel/150x150 accessed during the hours of 10:00 to 12:00 on 2019-01-23 02:39:46?']
['The status was success.', '2019-01-23 14:46:48.', '2019-01-23 16:28:11.', 'IP address 5.124.78.68 experienced success.', 'One time.']


In [13]:
inputs = tokenizer(input_list, return_tensors="pt", padding=True, truncation=True)

In [15]:
import torch

with torch.no_grad():
    outputs = model.generate(**inputs)

predictions = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1249: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [17]:
!pip install evaluate
!pip install sacrebleu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 8.5 MB/s eta 0:00:00


In [19]:
from evaluate import load

metric = load("sacrebleu")
references = [output for output in output_list]
results = metric.compute(predictions=predictions, references=references)
print(results)

{'score': 39.14546946035655, 'counts': [2601, 1635, 1032, 562], 'totals': [3265, 2765, 2265, 1765], 'precisions': [79.66309341500765, 59.13200723327306, 45.562913907284766, 31.841359773371106], 'bp': 0.7656156188269443, 'sys_len': 3265, 'ref_len': 4137}


In [20]:
predictions[0:5]

['The status was success.',
 '2019-01-23 12:49:42.',
 '2019-01-23 12:49:42.',
 'IP address 188.18.181.229 experienced success.',
 'One time.']

In [21]:
output_list[0:5]

['The status was success.',
 '2019-01-23 14:46:48.',
 '2019-01-23 16:28:11.',
 'IP address 5.124.78.68 experienced success.',
 'One time.']